# Classification using Machine Learning and the kNN algorithm


<img src="imgs/brain_learning.jpg">

<p style="text-align:left;">
    <a href="https://pixabay.com/illustrations/brain-chip-neurons-machine-learning-6010961/">Photo Credit: Pixabay</a><br>
    <span style="float:right;">
        April 5, 2023<br>
        Firas Moosvi
    </span>
</p>

In [12]:
from pandas import *
from altair import *
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler


def check_accuracy(*kwargs):

    score = accuracy_score(*kwargs)

    print(f"The model accuracy is {score:.1%}.")


trial = read_csv("data/trial_data.csv")

chart_trial = (
    Chart(trial)
    .mark_circle(size=50)
    .encode(
        X("dots:Q", title="Dots"),
        Y("vertices:Q", title="Number of Vertices"),
        Color("stroke:N", title="", legend=None),
    )
    .properties(
        title="Classifying a shape as Group 1 (Orange) or Group 0 (Blue)",
        width=600,
        height=250,
    )
    .configure_title(anchor="start", fontSize=24)
).configure_axis(labelFontSize=16, titleFontSize=20)

## Learning Context (3 mins)

<img src="imgs/avatars.jpg" width=50% align="center">

### Course Details

- Block 2 of the Master of Data Science program at UBCV
    - Situated in DSCI 571 - Supervised Learning I
    - Required course in the program
- **Prerequisite**:
    - DSCI 511 - Programming for Data Science
    - DSCI 521 - Computing Platforms for Data Science
- Almost everyone in here wants to be here and is excited to learn more!

### Programming Experience

- Least experienced: One full block (~5 weeks) of working in Python and R
- Most experienced:  Worked in software industry for 2+ years
- Everyone is familiar with course workflow
    - Watch a short video before every class
    - Jupyter Notebook "worksheet" during class with examples and demos
- ~ 120 students in the class

## Introduction (2 mins)

In today's class we will discuss how to classify data using Machine Learning, in particular using the $k$-nearest neighbours algorithm.

<img src ="imgs/knn.png" width=60%>

**Caption**: Pictoral description of the kNN algorithm, with three different classes (A, B, C) and a test point ($P_t$). The $k$ nearest neighbours of $P_t$ dictates which class the point belongs to. 

Source: Atallah, Dalia M., et al. [Predicting kidney transplantation outcome based on hybrid feature selection and KNN classifier](https://link.springer.com/article/10.1007/s11042-019-7370-5). Multimedia Tools and Applications 78 (2019): 20383-20407 {cite}`Atallah:2019`.

### Learning Intentions

After deliberate **review** of this lesson, consideration of supplementary content, and sufficient **practice**, I intend for students to be able to:

- Develop some intuition about classification using machine learning.

- Summarize the $k$NN algorithm and examine its strengths and weaknesses.

- Set up and use the $k$NN algorithm on a dataset.

- Critically evaluate the machine learning process and consider the importance of making human-centered choices.

### Pre-Lecture Video Review

Before you consume the content in this class, you should have watched the Pre-Lecture Video.
Here are the key points from that video that I'm expecting you to know:

#### Machine Learning in General

<img src="https://www.sap.com/dam/application/shared/graphics/what-is-machine-learning-process.svg" width=100%>

**Caption**: General workflow of the Machine Learning process. Image is copyright of [SAP Insights](https://www.sap.com/canada/insights/what-is-machine-learning.html), used under the education copyright exception.

#### Separating the training and testing data

- To determine accuracy, you **MUST** use data that the classifier has never seen before.

- Some people call this, the "**Golden Rule**" of Machine Learning:

```{hint} Golden Rule
The Test data cannot influence the training data in any way!
```

- Before you start working with the data, you should "split" the data into a "training set" and a "test set"

<img src="imgs/training_test.png" width=80%>

#### **Key Terms**

- **Features ($X$)**: "Input(s)" into the algorithm; features are measurable or observable values that are important attributes of the dataset (columns)
- **Target ($y$)**: "Output", is the feature we want to predict or derive from the machine learning algorithm.
- **Prediction**: After we create and train a machine learning model, a prediction is the result of deploying the model on test data.
- **Train set**: A subset of the full dataset that will be used to train the model, this dataset should have correct labels for the target.
- **Test data set**: A subset of the full dataset that will be used to assess model accuracy.
    
[Source: H2O.ai wiki](https://h2o.ai/wiki/)

## Activity: Our (toy) problem (3 mins): 

I have a series of **shapes**, and they have several **features**, and the goal is to *classify* a shape based on its features and put it in **Group 1** or **Group 0**.

- What "features" of the shape might be important?

- We are going to see a series of shapes, and your task is to categorize them as `Group 1` or `Group 0`.

- What "features" of the shape might be important?

- After each shape, I will tell you the correct answer (target).

<img src="imgs/shape.png" align="center" width=100%>

### Debrief

- Can you imagine a computer doing the same task?

| Feature Description | Column   |
|---------------------|----------|
| Number of vertices  | Vertices |
| Number of dots      | Dots     |
| Blob (1) or pointy (0) | Blob     |

| Target Description  | Target Column |
|---------------------|---------------|
| Group 1 or Group 0  | Stroke        |

Before we get into actually classifying, let's first visualize our full dataset.

### Load and Visualize Data

In [13]:
# First let's load our training data set
train = read_csv("data/train.csv")

train.head()

,dots,vertices,blob,stroke
0,28,2,0,0
1,25,6,1,0
2,28,18,0,1
3,39,7,0,1
4,38,2,1,0


Let's try to visualize it (if you are curious about the code to generate this, you can download the notebook and look at the code in `setup.ipynb`):

<img src="imgs/train.png" width=80%>

That's a lot of points!

Remember that there is still a third dimension (`blob` which has a value of `0` or `1`) that is not shown in the plot above, so keep that in mind going forward!

Let's try to formalize this and turn it into an algorithm that a computer can do for us.

## $k$ Nearest Neighbours (kNN) method (5 mins)

<img src="imgs/knn2.jpg" width=100%>

[Source: Natasha Latysheva, Cambridge Coding Academy](https://cambridgecoding.wordpress.com/2016/01/16/machine-learning-under-the-hood-writing-your-own-k-nearest-neighbour-algorithm/)

### Formalizing our Intuition

- What if we looked at the closest "neighbour" of the test point, and just classified based on the neighbouring point?

- How would we quantify the closest neighbour?
    - Euclidean Distance between point A, $P_A$($x_1$,$y_1$) and point B, $P_B$($x_2$,$y_2$) is:
       - $D = \sqrt{ (x_2 - x_1)^2 + (y_2 - y_1)^2) }$

<img src="imgs/euclidean2.png" width=60% align="center">

**Caption**: Definition of the 2D Euclidean distance for the shortest distance between two points $P_A$($x_1$,$y_1$) and point B, $P_B$($x_2$,$y_2$).

- To calculate the Euclidean distance between two points in three dimensions with points , we just extend the 2D formula.

- For two points in 3D space, $P_C$($x_1$,$y_1$,$z_1$) and $P_D$($x_2$,$y_2$,$z_2$):

    - $d = \sqrt{ (x_2 - x_1)^2 + (y_2 - y_1)^2 + (z_2 - z_1)^2}$


- And so on... for points in N-dimensions

## Challenges with the KNN method (5 mins)

The KNN algorithm is very intuitive, but it has some severe limitations.

### Can you think of some problems with the KNN method?

- What if there's a tie?
- Computationally intensive 
- Scale



### Problem 1: Scaling

- Points that look equidistant will not have the same distances unless the data are re-scaled.
    

- **Solution: Re-scale the data!**
    
    - To address this problem, we can re-scale the data by transforming the data so it has a mean of 0, and a standard deviation of 1.

    - We can do this by subtracting the mean $\bar{x}$, and then dividing by the standard deviation $\sigma$.

    $$
    Data_{scaled} = \frac{x - \bar{x}}{\sigma}
    $$

    - You may recognize this as a $z$-score

### Problem 2: Stability/outliers

- If our data is noisy, this method is prone to errors due to outliers and/or just random chance

- **Solution: Consider more than one neighbour!**
    - How many neighbours?
        - Choose an odd number so there are no ties!
        - Start with a low-number ($k_1 = 3$), run your classification on the training dataset, record accuracy; repeat for a bunch of $k$ values.
        - Plot the accuracy vs. $k$ and consider tradeoffs between bias (training error) and variance (testing error). See [here](https://medium.com/30-days-of-machine-learning/day-3-k-nearest-neighbors-and-bias-variance-tradeoff-75f84d515bdb) for a nice discussion of this.
        - Rough guideline: $k$ will depend on the size of your dataset. Probably not worth it to increase $k$ beyond $\approx \sqrt{N}$, where $N$ is the number of data points.

### Problem 3: Speed/Efficiency

- The Euclidean distances need to be re-calculated for every new test point against all points!

- **Solution:** No good solution - for extremely large datasets, computation of distances is very expensive!

### Problem 4: Curse of Dimensionality

- The kNN algorithm works reasonably well when the number of features ($d$) is small, but as $d$ increases, accuracy (and suitability) quickly falls !

- **Solution**: Not really a good solution here either, if $d$ is high you could do some dimensionality reduction, but other algorithms are probably better suited!

### Summary: Steps of the kNN algorithm

- Load Data and split into test/train sets
- Visualize Data
- Rescale Data
- Create classifier
- Train Algorithm
- Check accuracy of classifier
- Tweak Hyper-parameters ($k$)
- Retrain Algorithm
- Check accuracy of classifier
- etc...(iterate)

## Demo of k Nearest Neighbours algorithm (5 mins)

In [14]:
# First, we need to rescale our training data set (we'll keep the means and standard deviations for the test set)

train_mean = train.mean().to_dict()
train_std = train.std().to_dict()

# For each of the features, subtract the mean and divide by the standard deviation
for col in ["vertices", "dots", "blob"]:
    train[col] = (train[col] - train_mean[col]) / train_std[col]

train.head()

,dots,vertices,blob,stroke
0,-0.560732,-0.297568,-0.498312,0
1,-0.816171,0.526527,2.006299,0
2,-0.560732,2.998813,-0.498312,1
3,0.375878,0.732551,-0.498312,1
4,0.290732,-0.297568,2.006299,0


````{hint} Tip
Note that the `sklearn` package provides us with several convenience functions to accomplish the above task.

```
train = read_csv("data/train.csv")
scaler = StandardScaler().set_output(transform="pandas")
scaler.fit(train[["vertices", "dots", "blob"]])
scaler.transform(train[["vertices", "dots", "blob"]]).head()
```
Go ahead and give it a shot, make sure the output is the same!

````



In [18]:
# Then, let's create a classifier (model) - we'll set $k = 5$ to start with.

classifier = KNN(n_neighbors=5, metric="euclidean")

In [23]:
# Now, let's Train our model

classifier.fit(X=train[["vertices", "dots", "blob"]], y=train["stroke"])

KNeighborsClassifier(metric='euclidean')

In [24]:
# Let's use the classifier to predict each row of our training data set:

predictions = classifier.predict(train[["vertices", "dots", "blob"]])
predictions

array([0, 1, 1, ..., 1, 0, 1])

```{important}
Let's check the accuracy of these predictions to see how well we did (on our training data).
```

In [25]:
check_accuracy(train["stroke"], predictions)

The model accuracy is 70.2%.


```{attention}

Once you are confident that your model is setup correctly, the next step is to run the model on **previously unseen** data (the **test** set).
```

In [26]:
# So let's load the test data set

test = read_csv("data/test.csv")
test

,dots,vertices,blob,stroke
0,25,1,0,1
1,31,4,0,1
2,31,1,0,0
3,19,0,1,1
4,25,0,0,0
...,...,...,...,...
1051,53,1,0,0
1052,25,9,0,0
1053,22,0,1,0
1054,34,1,0,0


In [27]:
# We need to rescale our data *exactly* the same way we scaled our training data set

for col in ["vertices", "dots", "blob"]:
    test[col] = (test[col] - train_mean[col]) / train_std[col]

test.head()

,dots,vertices,blob,stroke
0,-0.816171,-0.503592,-0.498312,1
1,-0.305293,0.114479,-0.498312,1
2,-0.305293,-0.503592,-0.498312,0
3,-1.327049,-0.709616,2.006299,1
4,-0.816171,-0.709616,-0.498312,0


````{hint} Tip
Again, note the `sklearn` package provides us with several convenience functions to accomplish the above task.

```
test = read_csv("data/test.csv")
scaler.transform(test[["vertices", "dots", "blob"]])
```
Go ahead and give it a shot, make sure the output is the same!

````

In [28]:
# Let's use the classifier to predict each row of our TEST data set:

predictions = classifier.predict(test[["vertices", "dots", "blob"]])
predictions

array([0, 1, 0, ..., 0, 0, 0])

In [11]:
# And finally, check our accuracy
check_accuracy(test["stroke"], predictions)

The model accuracy is 65.6%.


## The importance of Context (5 mins)

- Let's start with the accuracy numbers:
    - Our algorithm gets it right ~ 66 times out of 100.
    - That's not bad - it's probably okay if we mis-classify some of them right?

- What if I told you that the data you've been working with has nothing to do with shapes, but people?

<img src="https://media3.giphy.com/media/8cPKrvyEpAMu2QFDWe/giphy.gif?cid=ecf05e47v152n2xlz4w3weap7880dy81u4rq4hs1zlds4o2q&rid=giphy.gif&ct=g" align="center">

- Here's what the data actually represents, all I did was change the names of the columns:

| Toy Column Name     | Actual Column Name                |
|---------------------|-----------------------------------|
| Vertices            | Prior Arrests                     |
| Dots                | Age                               |
| Blob                | Sex                               |
| Stroke              | Re-offense committed (recidivism) |

### The Context

> "The United States locks up far more people than any other country, a disproportionate number of them black. For more than two centuries, the key decisions in the legal process, from pretrial release to sentencing to parole, have been in the hands of human beings guided by their instincts and personal biases."

- [Machine Bias Risk Assessments in Criminal Sentencing](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing)

#### Risk Scores

> "If computers could accurately predict which defendants were likely to commit new crimes, the criminal justice system could be fairer and more selective about who is incarcerated and for how long. The trick, of course, is to make sure the computer gets it right. If it’s wrong in one direction, a dangerous criminal could go free. If it’s wrong in another direction, it could result in someone unfairly receiving a harsher sentence or waiting longer for parole than is appropriate."

- [Machine Bias Risk Assessments in Criminal Sentencing](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing)

#### COMPAS and Northpointe

COMPAS is one such proprietary software, developed by a for-profit company called Northpointe.

> Northpointe’s core product is a set of scores **derived from 137 questions that are either answered by defendants or pulled from criminal records**.



<img src="imgs/context.png" width=100%>

[Source: ProPublica](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing)

Here's how it performs:


<img src="imgs/propublica2.png" width=80%>

[Source: ProPublica](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing)



### Debrief

- Now let's re-evaluate the algorithm performance (65.6%) with this contextual information.

- When we changed the context, we started to look beyond numbers, and saw human beings instead.


- Consider your choices carefully - they may seem inoccuous initially, but hiding behind black box algorithms and deploying Weapons of Math Destruction ([Cathy O'Neil](https://en.wikipedia.org/wiki/Weapons_of_Math_Destruction)) without appropriately scrutinizing the underlying data is DANGEROUS!



- "Data" is not a panacea or a magical cure-all ; it's not biased in itself but it **does** reflect the biases, oppressions, systemic inequties within our society.

## Preview of next few classes (1 min)

Over the next few weeks in lectures and labs we will:

- carefully reproduce the Propublica analysis by treating the Northpointe algorithm as a binary classifier.

- analyze what it means for an algorithm to be "fair".

- look at Northpointe's response, and those of some other academics about this issue.

- consider metrics beyond simple "accuracy".

- learn some more sophisticated ML algorithms to see if we can remove bias (**Spoiler Alert: We will fail spectacularly!**)

## Summary of Learning Intentions (1 mins)

- Develop intuition about classification using machine learning.

- Identify the general steps of classification using machine learning.

- Summarize the kNN algorithm and examine its advantages and limitations.

- Critically evaluate the machine learning process and consider the importance of making human-centered choices.

### Review of KNN

<img src="imgs/knn2.jpg" width=100%>

[Source: Natasha Latysheva, Cambridge Coding Academy](https://cambridgecoding.wordpress.com/2016/01/16/machine-learning-under-the-hood-writing-your-own-k-nearest-neighbour-algorithm/)

### Acknowledgements

<img src="imgs/propublica.png" width=100%>

[Source: ProPublica](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing)


### Acknowledgements

- UC Berkeley Foundations of Data Science Data8 course notes, in particular [David Wagner's chapter on Classification](https://inferentialthinking.com/chapters/17/Classification.html).

- UBC's [Varada Kolhatkar and Mike Gelbart's CPSC 330 class notes on Supervised Learning Methods](https://github.com/UBC-CS/cpsc330/blob/master/lectures/04_kNNs-SVM-RBF.ipynb); some material from this lesson was adapted or inspired from this work.

- [ProPublica article](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing) that inspired this class.

- [Allen Downey's Recidivism Case Study](https://allendowney.github.io/RecidivismCaseStudy/index.html).

- UBC's [Mike Gelbart's video lectures on KNN](https://www.youtube.com/watch?v=JRF6oELLn0M&list=PLWmXHcz_53Q02ZLeAxigki1JZFfCO6M-b&index=4).

## Additional (Optional) Readings

- [Can the criminal justice system’s artificial intelligence ever be truly fair?](https://massivesci.com/articles/machine-learning-compas-racism-policing-fairness/)

<img src="imgs/incarceration.png" width=100%>

[Source: Pew Research](https://www.pewresearch.org/fact-tank/2021/08/16/americas-incarceration-rate-lowest-since-1995/)

- It is illegal in the United States for judges to sentence based on race, color, religion, sex, age.

- And yet...

<img src="imgs/lifetime-likelihood-of-imprisonment-by-race.png" width=100%>

[Source: The Sentencing Project](https://www.sentencingproject.org/criminal-justice-facts/)